# Úkol č. 2 - předzpracování dat a binární klasifikace (do 9. listopadu 23:59)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny vhodné k trénování modelu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí křížové validace. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor s predikcemi pro vyhodnocovací data (`results.csv`).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [3]:
import pandas as pd
import numpy as np
import math

# Data

In [4]:
data = pd.read_csv('data.csv')

In [5]:
data

,ID,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,0,1,3,"Dorking, Mr. Edward Arthur",male,19.00,0,0,A/5. 10482,8.0500,NaN,S,"England Oglesby, IL"
1,1,1,2,"Smith, Miss. Marion Elsie",female,40.00,0,0,31418,13.0000,NaN,S,NaN
2,2,0,3,"Hegarty, Miss. Hanora ""Nora""",female,18.00,0,0,365226,6.7500,NaN,Q,NaN
3,3,0,3,"Sage, Mr. John George",male,NaN,1,9,CA. 2343,69.5500,NaN,S,NaN
4,4,0,3,"Cacic, Miss. Marija",female,30.00,0,0,315084,8.6625,NaN,S,NaN
5,5,0,1,"Taussig, Mr. Emil",male,52.00,1,1,110413,79.6500,E67,S,"New York, NY"
6,6,0,3,"Flynn, Mr. John",male,NaN,0,0,368323,6.9500,NaN,Q,NaN
7,7,1,1,"Taylor, Mrs. Elmer Zebley (Juliet Cummins Wright)",female,NaN,1,0,19996,52.0000,C126,S,"London / East Orange, NJ"
8,8,0,2,"Berriman, Mr. William John",male,23.00,0,0,28425,13.0000,NaN,S,"St Ives, Cornwall / Calumet, MI"
9,9,0,2,"Abelson, Mr. Samuel",male,30.00,1,0,P/PP 3381,24.0000,NaN,C,"Russia New York, NY"


In [58]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
ID           1000 non-null int64
survived     1000 non-null int64
pclass       1000 non-null int64
name         1000 non-null object
sex          1000 non-null object
age          797 non-null float64
sibsp        1000 non-null int64
parch        1000 non-null int64
ticket       1000 non-null object
fare         1000 non-null float64
cabin        226 non-null object
embarked     998 non-null object
home.dest    554 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 101.6+ KB


None

In [59]:
data.isnull().sum(axis=0)

ID             0
survived       0
pclass         0
name           0
sex            0
age          203
sibsp          0
parch          0
ticket         0
fare           0
cabin        774
embarked       2
home.dest    446
dtype: int64

In [60]:
display(data.nunique())

ID           1000
survived        2
pclass          3
name          999
sex             2
age            93
sibsp           7
parch           8
ticket        740
fare          248
cabin         156
embarked        3
home.dest     308
dtype: int64

In [61]:
data1 = data.copy()

Vzhledem k tomu, že strom pracuje s číselnými hodnotami, tak je potřeba poupravit příznak s pohlavím, ten upravíme tak, že ho převedeme na categorii a následně uložíme jeho kód ###
male -> 0 a female -> 1

In [62]:
data1['sex'] = data1['sex'].astype('category').cat.codes

In [63]:
data2 = data1.copy()

Cabin udava cislo kajuty, ale je zde obsazena i informace o palube, ktera je pro nas podle me vice uzitecna, proto si ponechame jen tuto informaci a nasledne ji take zakodujeme, jako jsme to udelali s pohlavim

In [64]:
data2['cabin'] = data2['cabin'].str.slice(stop=1)

In [65]:
data2['cabin'] = data2['cabin'].astype('category').cat.codes

In [66]:
data2.isnull().sum(axis=0)

ID             0
survived       0
pclass         0
name           0
sex            0
age          203
sibsp          0
parch          0
ticket         0
fare           0
cabin          0
embarked       2
home.dest    446
dtype: int64

# chybejici hodnoty

potrebujeme doplnit jen hodnoty, ktere se tykaji veku, jelikoz ostatni priznaky nebudeme vyuzivat

Hodnoty bych doplnil tak, ze bych nejdrive vzal prumer pro jendotliva pohlavi, v kazde tride a zda prezila nebo ne, a podle tohoto bych chybejici data doplnil.
Rozhodne by se zde nabizela i metoda nejblizsich sousedu.

Nan prepíšeme -1, rozhodl jsem se tak, protože kdybych se snazil chybející hodnoty nejak doplnit, tak zde vytvořím informační šum, takhle je naprosto jasné, že příznak neznáme, tudíž strom tolik neovlivní

In [11]:
data2 = data2.replace(np.nan, -1)

Data rozdelime v pomeru 75-25% na trenovaci a validacni

In [12]:
data2_train, data2_vali = np.split(data2.sample(frac=1, random_state=1729), 
                                   [int(0.75 * len(data2))])

rozhodl jsem se vyuzit priznaky : trida, pohlavi, vek - ktere podle me budou mit nejvetsi vliv, dale palubu a informace o rodine, jak je videt nize

In [13]:
Xdata_train = data2_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'cabin']]
Ydata_train = data2_train['survived']
Xdata_vali = data2_vali[['pclass', 'sex', 'age', 'sibsp', 'parch', 'cabin']]
Ydata_vali = data2_vali['survived']

# Strom

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
dt = DecisionTreeClassifier()
dt.get_params()

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [16]:
#naucime na nasich datech
dt.fit(Xdata_train, Ydata_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

metrics pouzivam na zjisteni uspesnosti naseho modelu

In [17]:
import sklearn.metrics as metrics 

podle toho, jak je presnost rozdilna, zaroven dosti dobra na trenovacich datech, je strom nejspise preuceny

In [18]:
print('acur score train : {0:.3f}'.format(metrics.accuracy_score(Ydata_train, dt.predict(Xdata_train))))
print('acur score vali : {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, dt.predict(Xdata_vali))))

acur score train : 0.909
acur score vali : 0.792


rozhodl jsem se trenovat tyto parametry, prootoze si myslim, ze hloubka bude mit na presnost stromu asi nejvetsi vliv, min_samples_split se moc nepovedl, protoze se ukaze, ze defaultni nastaveni je nejlepsi, to same u kriteria, a dalsi z moznych hyperparametru mi neprisli, jako, ze by nas strom nejakym zpusobem vylepsili

In [19]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'max_depth': range(1,20), 
    'criterion': ['entropy', 'gini'], 
    'min_samples_split': range(2,5),
}
param_comb = ParameterGrid(param_grid)

In [20]:
val_acc = []
train_acc = []
for params in param_comb:
    dt = DecisionTreeClassifier(max_depth=params['max_depth'], criterion=params['criterion'], min_samples_split=params['min_samples_split'])
    dt.fit(Xdata_train, Ydata_train)
    train_acc.append(metrics.accuracy_score(Ydata_train, dt.predict(Xdata_train)))
    val_acc.append(metrics.accuracy_score(Ydata_vali, dt.predict(Xdata_vali)))

zde uz je videt, ktere hyperparametry se pro nas jevi jako nejlepsi

In [21]:
dt_best_params = param_comb[np.argmax(val_acc)]

## vysledny strom

In [22]:
dt = DecisionTreeClassifier(**dt_best_params)
dt.fit(Xdata_train, Ydata_train)
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, dt.predict(Xdata_train))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, dt.predict(Xdata_vali))))

accuracy score (train): 0.861
accuracy score (validation): 0.824


# Les

prozatim stejny proces jako se stromem

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [24]:
rf = RandomForestClassifier()

In [25]:
rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [26]:
rf.fit(Xdata_train, Ydata_train)

C:\Users\pavel\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

stejny problem s preucenim

In [27]:
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, rf.predict(Xdata_train))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, rf.predict(Xdata_vali))))

accuracy score (train): 0.892
accuracy score (validation): 0.776


zde zkousim jeste ladit hyperparametr n_estimators ktery nam udava pocet stromu, ktery les ma

In [28]:
param_grid = {
    'max_depth': range(1,50), 
    'criterion': ['entropy', 'gini'],
    'n_estimators': range(2,10)
}
param_comb = ParameterGrid(param_grid)

In [29]:
val_acc = []
train_acc = []
for params in param_comb:
    rf = RandomForestClassifier(max_depth=params['max_depth'], criterion=params['criterion'], n_estimators=params['n_estimators'])
    rf.fit(Xdata_train, Ydata_train)
    train_acc.append(metrics.accuracy_score(Ydata_train, rf.predict(Xdata_train)))
    val_acc.append(metrics.accuracy_score(Ydata_vali, rf.predict(Xdata_vali)))

takze nejlepsi vysledek jsem dostal pro 4 stromy s maximalni hloubkou 6 za pouziti entropie

In [30]:
best_params = param_comb[np.argmax(val_acc)]
print(best_params)
print(val_acc[np.argmax(val_acc)])
print(train_acc[np.argmax(val_acc)])

{'n_estimators': 3, 'max_depth': 35, 'criterion': 'entropy'}
0.832
0.88


## Vysledny les

In [31]:
rf = RandomForestClassifier(**best_params)
rf.fit(Xdata_train, Ydata_train)
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, rf.predict(Xdata_train))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, rf.predict(Xdata_vali))))

accuracy score (train): 0.873
accuracy score (validation): 0.768


# AdaBoost

In [32]:
from sklearn.ensemble import AdaBoostClassifier

In [33]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)

In [34]:
clf.get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 100,
 'random_state': 0}

In [35]:
clf.fit(Xdata_train, Ydata_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

tohle nevypada zle, ale uspesnost nic moc

In [36]:
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, clf.predict(Xdata_train))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, clf.predict(Xdata_vali))))

accuracy score (train): 0.801
accuracy score (validation): 0.780


In [37]:
param_grid = {
    'n_estimators': range(60,100,3),
    'learning_rate': [0.1, 0.5, 1]
}

param_comb = ParameterGrid(param_grid)

val_acc = []
train_acc = []
for params in param_comb:
    clf = AdaBoostClassifier(n_estimators=params['n_estimators'], learning_rate=params['learning_rate'])
    clf.fit(Xdata_train, Ydata_train)
    train_acc.append(metrics.accuracy_score(Ydata_train, clf.predict(Xdata_train)))
    val_acc.append(metrics.accuracy_score(Ydata_vali, clf.predict(Xdata_vali)))

bohuzel je videt, ze se uspesnost moc nezvedla

In [38]:
best_params = param_comb[np.argmax(val_acc)]
print(best_params)
print(val_acc[np.argmax(val_acc)])
print(train_acc[np.argmax(val_acc)])

{'n_estimators': 75, 'learning_rate': 0.5}
0.804
0.7893333333333333


## Vysledny AdaBoost

In [39]:
clf =  AdaBoostClassifier(**best_params)
clf.fit(Xdata_train, Ydata_train)
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, clf.predict(Xdata_train))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, clf.predict(Xdata_vali))))

accuracy score (train): 0.789
accuracy score (validation): 0.804


Nejlepe tedy dopadl strom o 2%

# KNN

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

In [41]:
knn = KNeighborsClassifier()

In [42]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [43]:
knn.fit(Xdata_train, Ydata_train)
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, knn.predict(Xdata_train))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, knn.predict(Xdata_vali))))

accuracy score (train): 0.797
accuracy score (validation): 0.728


In [44]:
param_grid = {
    'n_neighbors': range(1,10),
    'p': [1, 2, 3]
}

param_comb = ParameterGrid(param_grid)

val_acc = []
train_acc = []
for params in param_comb:
    knn = KNeighborsClassifier(n_neighbors=params['n_neighbors'], p=params['p'])
    knn.fit(Xdata_train, Ydata_train)
    train_acc.append(metrics.accuracy_score(Ydata_train, knn.predict(Xdata_train)))
    val_acc.append(metrics.accuracy_score(Ydata_vali, knn.predict(Xdata_vali)))

In [45]:
best_params = param_comb[np.argmax(val_acc)]

In [46]:
knn = KNeighborsClassifier(**best_params)
knn.fit(Xdata_train, Ydata_train)
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, knn.predict(Xdata_train))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, knn.predict(Xdata_vali))))

accuracy score (train): 0.796
accuracy score (validation): 0.764


In [47]:
scaler = MinMaxScaler()
Xtrain_scaled = scaler.fit_transform(Xdata_train)

Xval_scaled = scaler.transform(Xdata_vali)

param_grid = {
    'n_neighbors': range(3,7),
    'p': [1, 2]
}

param_comb = ParameterGrid(param_grid)

val_acc = []
train_acc = []
for params in param_comb:
    knn = KNeighborsClassifier(n_neighbors=params['n_neighbors'], p=params['p'])
    knn.fit(Xtrain_scaled, Ydata_train)
    train_acc.append(metrics.accuracy_score(Ydata_train, knn.predict(Xtrain_scaled)))
    val_acc.append(metrics.accuracy_score(Ydata_vali, knn.predict(Xval_scaled)))

best_params = param_comb[np.argmax(val_acc)]

knn = KNeighborsClassifier(**best_params)
knn.fit(Xtrain_scaled, Ydata_train)
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, knn.predict(Xtrain_scaled))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, knn.predict(Xval_scaled))))

accuracy score (train): 0.803
accuracy score (validation): 0.804


## Vysledny KNN

In [48]:
knn = KNeighborsClassifier(**best_params)
knn.fit(Xtrain_scaled, Ydata_train)
print('accuracy score (train): {0:.3f}'.format(metrics.accuracy_score(Ydata_train, knn.predict(Xtrain_scaled))))
print('accuracy score (validation): {0:.3f}'.format(metrics.accuracy_score(Ydata_vali, knn.predict(Xval_scaled))))

accuracy score (train): 0.803
accuracy score (validation): 0.804


### to neni spatne ale strom byl stale lepsi

# cross-validation

In [49]:
from sklearn.model_selection import KFold, LeaveOneOut
import math

In [50]:
n_splits = 5

In [51]:
random_seed = 12247112

In [53]:
err = []
comb_err = []
    # Pomocí KFold rozdělíme index na trénovací a validační
for train, val in KFold(n_splits=n_splits, random_state=random_seed).split(data2.index):
    
    
    Xtrain = data2[data2.index.isin(train)][['pclass', 'sex', 'age', 'sibsp', 'parch', 'cabin']]
    ytrain = data2[data2.index.isin(train)]['survived']
    Xval = data2[data2.index.isin(val)][['pclass', 'sex', 'age', 'sibsp', 'parch', 'cabin']]
    yval = data2[data2.index.isin(val)]['survived']
    
    # Natrénujeme model a změříme validační chybu (RMSLE)
    dt = DecisionTreeClassifier(**dt_best_params)
    dt.fit(Xtrain, ytrain)
    comb_err.append(np.sqrt(metrics.mean_squared_error(yval, dt.predict(Xval))))
        
# Zprůměrujeme chybu
err.append(np.mean(comb_err))

In [54]:
best_index = np.argmin(err)
best_err = err[best_index]
print('Best RMSLE: ', best_err)

Best RMSLE:  0.4839712549395271


# vysledek

In [ ]:
evaluation = pd.read_csv('evaluation.csv')

data je potreba transformovat jako puvodni na kterych se model ucil

In [ ]:
evalu = evaluation[['pclass', 'sex', 'age', 'sibsp', 'parch', 'cabin']]

In [ ]:
evalu['sex'] = evalu['sex'].astype('category').cat.codes
evalu['cabin'] = evalu['cabin'].str.slice(stop=1)
evalu['cabin'] = evalu['cabin'].astype('category').cat.codes
evalu = evalu.replace(np.nan, -1)

In [ ]:
predict = dt.predict(evalu)

In [ ]:
evaluation['predict'] = predict 

In [ ]:
results = evaluation[['ID', 'predict']]

In [ ]:
results.to_csv('results.csv',index=False)